<a href="https://colab.research.google.com/github/yash-1708/Stock-Analyst-Rating/blob/master/scrapingmarketsmojoBIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
from dateutil import parser
import sqlalchemy
from datetime import datetime

In [2]:
driver = webdriver.Chrome("chromedriver.exe")
broker=[] #List to store name of the broker
company_name=[] #List to store company names 
current_price=[] #List to store rating of the product
predict_date=[]
recomended_buying=[]
target_price=[]
driver.get("https://www.marketsmojo.com/recommendation/")


In [3]:
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')
Alldata=soup.select_one("div.righttexcontent2 div.right-table01 div.scr-result2")
# for row in Alldata.findAll('div',class_='middle-colum02'):
for row in Alldata.findAll('div',attrs={'class':'middle-colum02'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)
    #predict_date.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott"))
     
for row in Alldata.findAll('div',attrs={'class':'middle-columactive'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)

Test Cell

In [4]:
temp_data      #test

'23 Oct 19'

Classifying into different dataframes

In [5]:
df1 = pd.DataFrame({'broker':broker})  
df2 = pd.DataFrame({'company_name':company_name})
df3 = pd.DataFrame({'current_price':current_price})
df4 = pd.DataFrame({'recomended_buing':recomended_buying})
df5=pd.DataFrame({'target_price':target_price})
df6=pd.DataFrame({'predict_date':predict_date})
#dfALL = pd.DataFrame(broker,company_name,current_price,recomended_buying,target_price)


Cleaning Data

In [6]:
#splitting operation
rb=df4['recomended_buing'].str.split('Rs.').str[1] 
cp=df3['current_price'].str.split('Rs.').str[1]
tp=df5['target_price'].str.split('Rs.').str[1]
predict_date=df6['predict_date'].str.split('10').str[0]



In [7]:
dfall=pd.DataFrame({'broker':broker, 'company_name':company_name, 'current_price':cp, 'recomended_buying':rb, 'predict_date':predict_date, 'target_price':tp})
dfallclean = dfall[dfall['predict_date']!='']
dfallclean['predict_date'] =  pd.to_datetime(dfallclean['predict_date'], format='%d %b %y')

In [8]:
dfallclean

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Prabhudas Lilladher,Engineers India,105.65,111.00,2019-11-08,156.00
1,Prabhudas Lilladher,Infosys,708.45,718.53,2019-11-07,819.00
2,Kotak Securities,Gujarat Gas,196.3,194.65,2019-11-07,213.00
3,Prabhudas Lilladher,Sun Pharma Inds.,421.8,435.34,2019-11-07,396.00
4,Prabhudas Lilladher,Hindalco,204,201.02,2019-11-07,215.00
5,Prabhudas Lilladher,Eris Lifesciences,417.65,422.67,2019-11-07,459.00
6,Geojit Financial Services,Exide Inds,186.65,190.34,2019-11-07,209.00
7,Prabhudas Lilladher,Tata Steel,397.8,398.87,2019-11-07,350.00
8,Prabhudas Lilladher,HCL Tech.,1150.1,1143.74,2019-11-07,1258.00
9,Prabhudas Lilladher,Kalpataru PowerTrans,450.5,439.31,2019-11-07,662.00


In [9]:
dfallclean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285 entries, 0 to 284
Data columns (total 6 columns):
broker               285 non-null object
company_name         285 non-null object
current_price        285 non-null object
recomended_buying    285 non-null object
predict_date         285 non-null datetime64[ns]
target_price         285 non-null object
dtypes: datetime64[ns](1), object(5)
memory usage: 15.6+ KB


In [10]:
database_username = 'root'
database_password = 'subodh30oct'
database_ip       = '127.0.0.1'
database_name     = 'final'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [11]:
maxdate = pd.read_sql('SELECT MAX(predict_date) FROM newmarketreco', con=database_connection)

In [12]:
maxdate

,MAX(predict_date)
0,2019-09-17


In [13]:
a=maxdate.loc[0,'MAX(predict_date)']

In [14]:
date_object1=maxdate['MAX(predict_date)']
date_object1.str
mynew=str(a)

In [15]:
print(type(mynew))

<class 'str'>


In [16]:
date_object1 = datetime.strptime(mynew, '%Y-%m-%d').date()

In [17]:
date_object1

datetime.date(2019, 9, 17)

In [18]:
mask = (dfallclean['predict_date'] > date_object1) 

C:\Users\dhruv\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [19]:
dfentry = dfallclean.loc[mask]
dfentry

,broker,company_name,current_price,recomended_buying,predict_date,target_price
0,Prabhudas Lilladher,Engineers India,105.65,111.00,2019-11-08,156.00
1,Prabhudas Lilladher,Infosys,708.45,718.53,2019-11-07,819.00
2,Kotak Securities,Gujarat Gas,196.3,194.65,2019-11-07,213.00
3,Prabhudas Lilladher,Sun Pharma Inds.,421.8,435.34,2019-11-07,396.00
4,Prabhudas Lilladher,Hindalco,204,201.02,2019-11-07,215.00
5,Prabhudas Lilladher,Eris Lifesciences,417.65,422.67,2019-11-07,459.00
6,Geojit Financial Services,Exide Inds,186.65,190.34,2019-11-07,209.00
7,Prabhudas Lilladher,Tata Steel,397.8,398.87,2019-11-07,350.00
8,Prabhudas Lilladher,HCL Tech.,1150.1,1143.74,2019-11-07,1258.00
9,Prabhudas Lilladher,Kalpataru PowerTrans,450.5,439.31,2019-11-07,662.00


In [21]:

dfentry.to_sql(con=database_connection, name='marketsmojorecos', if_exists='append')